## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
appl_df = application_df.drop(['EIN', 'NAME'], axis=1)
appl_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
appl_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_ct = appl_df['APPLICATION_TYPE'].value_counts()
app_ct

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_ct[app_ct < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    appl_df['APPLICATION_TYPE'] = appl_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
appl_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_ct = appl_df['CLASSIFICATION'].value_counts()
class_ct

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C8210        1
C3700        1
C2380        1
C1820        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#class_ct = appl_df['CLASSIFICATION'].value_counts(counts > 1)

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_ct[class_ct < 250].index)

# Replace in dataframe
for cls in classifications_to_replace:
    appl_df['CLASSIFICATION'] = appl_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
appl_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_numbers = pd.get_dummies(appl_df)
charity_numbers

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = charity_numbers['IS_SUCCESSFUL']
X = charity_numbers.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Import an Adaptive Boosting classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
clf = AdaBoostClassifier(random_state=1).fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

print(classification_report(y_test, y_pred))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

              precision    recall  f1-score   support

           0       0.71      0.66      0.68      4003
           1       0.72      0.76      0.74      4572

    accuracy                           0.71      8575
   macro avg       0.71      0.71      0.71      8575
weighted avg       0.71      0.71      0.71      8575

Training Score: 0.7232156740786814
Testing Score: 0.714868804664723


## Compile, Train and Evaluate the Model

I adjusted the hidden layers a few times, from 2 layers @ 8/16, to 3 layers 11/15/22. The additional layers produced a better outcome in terms of accuracy, but still not the 75% I hoped for.

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  11
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 22

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, activation="relu", input_dim=45))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer3, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 11)                506       
                                                                 
 dense_3 (Dense)             (None, 15)                180       
                                                                 
 dense_4 (Dense)             (None, 22)                352       
                                                                 
 dense_5 (Dense)             (None, 1)                 23        
                                                                 
Total params: 1,061
Trainable params: 1,061
Non-trainable params: 0
_________________________________________________________________


In [29]:
#!pip install tf-nightly

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\msfte\\anaconda3\\envs\\PythonData\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.




  Created wheel for jax: filename=jax-0.3.25-py3-none-any.whl size=1308507 sha256=328fd1ad2e3f777738dcc667228d6a7a90d36c1384553d7d896f1cdd1a162ed7
  Stored in directory: c:\users\msfte\appdata\local\pip\cache\wheels\a5\79\44\dbae745c48940291d6d4d157f7faa6f0b27f198781073d8c78
Successfully built jax


I attempted to use the Adadelta and AdamW optimization here as well; I wanted to address weight decay and both do this. However, the AdamW is experimental and when I pipinstalled the module for applying this optimization, it gave me an error that wasn't resolvable. When I attempted the Adadelta optimizer, the model immediately fit to a 0.5592 loss: 0.7433 accuracy throughout every single epoch, which clearly wasn't the expected outcome, so I discarded that optimize and went back to the Adam optimizer instead. ML is iterative and requires a considerable amount of trial-and-error.

In [36]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

So far, I have attempted to adjust the epochs 5 times: 50, 100, 200, 300, 250. 200 is probably optimal; anything above this seems to create decay.

In [37]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.5324 - accuracy: 0.7412
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5322 - accuracy: 0.7419
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5325 - accuracy: 0.7423
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5325 - accuracy: 0.7420
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5328 - accuracy: 0.7415
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5328 - accuracy: 0.7423
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5329 - accuracy: 0.7415
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7418
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5325 - accuracy: 0.7420
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5330 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5323 - accuracy: 0.7421
Epoch 82/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5327 - accuracy: 0.7414
Epoch 83/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5327 - accuracy: 0.7421
Epoch 84/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.7421
Epoch 85/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5325 - accuracy: 0.7420
Epoch 86/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7419
Epoch 87/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7419
Epoch 88/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7414
Epoch 89/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5321 - accuracy: 0.7417
Epoch 90/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 

804/804 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7421
Epoch 161/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7428
Epoch 162/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7427
Epoch 163/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5322 - accuracy: 0.7421
Epoch 164/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7423
Epoch 165/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5320 - accuracy: 0.7424
Epoch 166/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7421
Epoch 167/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7421
Epoch 168/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7421
Epoch 169/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5324 - a

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5672 - accuracy: 0.7257 - 373ms/epoch - 1ms/step
Loss: 0.5671572089195251, Accuracy: 0.7257142663002014


In [ ]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharityboosted.h5')

In [ ]:
# save charity_numbers as a .csv
charity_numbers.to_csv('charity_numbers_boosted.csv')